In [ ]:
# Experimental Topology
# lockin AO 2 -> Analog Attenuator Vdd
# lockin AO 3 -> Analog Attenuator Vc (Control Voltages)
# Keithley Output -> Voltage divider with 1 MOhm, DAQmx AI1
# AWG Sync Marker Out -> DAQmx PFI0
# AWG Samp. Marker Out -> PSPL Trigger

In [1]:
cd ..

C:\Users\qlab\Documents\GitHub\pycontrol


In [2]:
from pycontrol.filters.io import WriteToHDF5, ProgressBar
import numpy as np
import itertools
import time
import matplotlib.pyplot as plt
import analysis.switching as sw
from adapt import refine
from analysis.h5shell import h5shell

In [3]:
from experiments.Pulse_Switching_CSHE import SwitchingExperiment

In [4]:
exp = SwitchingExperiment()
exp.sample = "CSHE2-C4R2"
exp.comment = "Phase Diagram -  AP to P - Interations = 3"
exp.polarity = 1 # 1: AP to P; -1: P to AP
exp.iteration = 3
exp.field.value = 0.0133
exp.measure_current = 3e-6
exp.init_instruments()

In [5]:
wr = WriteToHDF5("data\CSHE-Switching\CSHE-Die2-C4R2\CSHE2-C4R2-AP2P_2016-06-29.h5")
pbar = ProgressBar(num=1, notebook=True)
edges = [(exp.daq_buffer, wr.data), (exp.daq_buffer, pbar.data)]
# edges = [(exp.daq_buffer, wr.data)]
exp.set_graph(edges)

In [6]:
coarse_ts = 1e-9*np.linspace(0.1, 5, 5) # List of durations
coarse_vs = np.linspace(0.3, 0.7, 5)
points    = [coarse_ts, coarse_vs]
points    = list(itertools.product(*points))

main_sweep = exp.add_unstructured_sweep([exp.pulse_duration, exp.pulse_voltage], points)

In [7]:
figs = []
t1 = time.time()
for i in range(exp.iteration):
    exp.reset()
    exp.run_sweeps()
    points, mean = sw.load_switching_data(wr.filename)
    figs.append(sw.phase_diagram_mesh(points, mean, title="Iteration={}".format(i)))
    new_points = refine.refine_scalar_field(points, mean, all_points=False,
                                criterion="integral", threshold = "one_sigma")
    if new_points is None:
        print("No more points can be added.")
        break
    #
    print("Elapsed time: {} min.".format((time.time()-t1)/60))
    main_sweep.update_values(new_points)

C:\Users\qlab\Anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:821: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  n_jobs=self.n_jobs)


11 new points added.
Added 11 new points.
Elapsed time: 0.22328650951385498

11 new points added.
Added 11 new points.
Elapsed time: 0.3343453884124756

20 new points added.
Added 20 new points.
Elapsed time: 0.44807287454605105


In [8]:
t2 = time.time()
# Shut down
exp.shutdown_instruments()
# For evaluation of adaptive method, plot the mesh
mesh, scale_factors = sw.scaled_Delaunay(points)
fig_mesh = sw.phase_diagram_mesh(points, mean, shading='gouraud')
plt.triplot(mesh.points[:,0]/scale_factors[0],
            mesh.points[:,1]/scale_factors[1], mesh.simplices.copy());
plt.show()